Стандартная версия Нужно реализовать rest api на базе flask в google colab.

выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
Реализовать ноутбук с сервером
Реализовать ноутбук с клиентом

In [1]:
!pip install dill

In [2]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [3]:
df = pd.read_csv("./Phishing_Email.csv")
df.head(3)

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email


In [4]:
df['Email Type'].value_counts()

Safe Email        11322
Phishing Email     7328
Name: Email Type, dtype: int64

In [5]:
df = df.replace({'Email Type':{'Safe Email':0, 'Phishing Email':1}})

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Email Type'],
                                                    test_size=0.33, random_state=42)
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [7]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [8]:
features = 'Email Text'
target = 'Email Type'

In [9]:
Email_Text = Pipeline([
                ('imputer', TextImputer('Email Text', '')),
                ('selector', ColumnSelector(key='Email Text')),
                ('tfidf', TfidfVectorizer())
            ])
feats = FeatureUnion([('Email Text', Email_Text),
                      ])

In [10]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: user 6.99 s, sys: 1.78 s, total: 8.77 s
Wall time: 9.91 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Email Text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Email '
                                                                                  'Text',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Email '
                                                                                     'Text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

In [11]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('Email Text',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='Email Text',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='Email '
                                                                      'Text')),
                                                  ('tfidf',
                                                   TfidfVectorizer())]))])),
 ('classifier', LogisticRegression())]

In [12]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [13]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [14]:
X_test.head(3)

,Unnamed: 0,Email Text,Email Type
0,14470,"Once upon a time, Harri wrote :> Title page ha...",0
1,3014,new books : generative studies we would like t...,0
2,16942,new book : japanese linguistics japanese lingu...,0


In [15]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [16]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Email Text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Email '
                                                                                  'Text',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Email '
                                                                                     'Text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

In [17]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [18]:
preds[:10]

array([0.03480584, 0.08468372, 0.05567492, 0.38078039, 0.88818018,
       0.14505569, 0.07654292, 0.0172681 , 0.92820363, 0.17128533])

In [19]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4727210505766423, F-Score=0.959, Precision=0.951, Recall=0.967


In [20]:
!pip3 install flask-ngrok

In [21]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [22]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 25vEpcJ5Ih4vlUp4thEZ9sEA6ZU_3Bnu17gKacRXhF6hLeefc
!./ngrok http 80

--2023-07-23 21:08:41--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.1’

ngrok-stable-linux- 100%[===================>]  13.21M  --.-KB/s    in 0.07s   

2023-07-23 21:08:41 (199 MB/s) - ‘ngrok-stable-linux-amd64.tgz.1’ saved [13856790/13856790]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [23]:
# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5656-35-245-112-149.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


In [24]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [25]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [26]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    Email_Text = ""

    request_json = request.get_json()

    if request_json["Email Text"]:
        Email_Text = request_json['Email Text']


    #print(Email_Text)
    preds = model.predict_proba(pd.DataFrame({"Email Text": [Email_Text]
                                              }))
    data["predictions"] = preds[:, 1][0]
    data["Email Text"] = Email_Text
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://bbc3-35-245-112-149.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Jul/2023 21:09:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2023 21:09:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2023 21:10:23] "POST /predict HTTP/1.1" 200 -


OK
